In [44]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import numpy as np
import laspy, lazrs
import random as random
import networkx as nx
from scipy import spatial
from pathlib import Path

# file paths
random_points_path = "random_points.csv"
subsets_path = "subsets"
subset_name = "sub"

In [3]:
points = pd.read_csv(random_points_path)

In [7]:
# generate a random point from the "points" dataframe
minDistance = 800
start_index = random.randint(0, points.shape[0])
end_index = random.randint(0, points.shape[0])
start = points.loc[start_index]
end = points.loc[end_index]

# get the distance between two points
def distance(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2 + (p1.z - p2.z)**2)

while (distance(start, end) < minDistance):
    start_index = random.randint(0, points.shape[0])
    end_index = random.randint(0, points.shape[0])
    start = points.loc[start_index]
    end = points.loc[end_index]

print("Start point: ", start["x"], start["y"], start["z"])
print("End point: ", end["x"], end["y"], end["z"])
print("Start index: ", str(start_index))
print("End index: ", str(end_index))

Start point:  1457.4500000011176 44.38000000081957 9.839999999999918
End point:  424.2000000011176 910.960000000894 36.22000000000003
Start index:  1722084
End index:  55821


In [45]:
# Hyperparameters and variables

# number of edges per node in the graph
depth = 4

# number of subsets/subgraphs (ideally subsets == resolution)
subsets = 20

# higher resolution = less points. effective resolution = subsets/resolution
resolution = 10000

# max tolerated slope angle (degrees)
max_slope = 40

# slope importance factor
slope_factor = 0.005

# height importance factor
height_factor = 1000

# heuristic function: greater return value means greater cost for the path (best path has low cost)
def heur(x1, y1, z1, x2, y2, z2):
    # print(f"distance weight: {math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)}")
    # print(f"slope angle: {slope_angle(x1, y1, z1, x2, y2, z2)}")
    # print(f"slope weight: {slope_factor * slope_angle(x1, y1, z1, x2, y2, z2)}")
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + height_factor * (z2 - z1) ** 2)
    #  + slope_factor * slope_angle(x1, y1, z1, x2, y2, z2)

def heur_dist(x1, y1, z1, x2, y2, z2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# other variables
skip = len(points) // subsets
print(f"number of points per subset: {skip}")
print(f"effective resolution: {subsets/resolution}")
print(f"number of subsets: {subsets}")

# plot variables
boldness = 1
plotSize = 10

# helper methods
def slope_angle(x1, y1, z1, x2, y2, z2):
    # v dot w = |v||w|cos(theta)
    # theta = arccos(v dot w / |v||w|)
    v = np.array([abs(x2 - x1), abs(y2 - y1), abs(z2 - z1)])
    w = np.array([0, 0, 1])
    v_norm = np.linalg.norm(v)
    if v_norm == 0:
        return 0
    return abs(math.degrees(math.acos(np.dot(v, w) / v_norm)))

number of points per subset: 558100
effective resolution: 0.002
number of subsets: 20


In [46]:
skip = len(points) // subsets  # Number of points per subset
points_per_subset = len(points) // resolution  # Number of points per subset based on resolution

output_dir = Path(subsets_path)
output_dir.mkdir(exist_ok=True)

for i in range(subsets):
    # Select the subset of points with points_per_subset points
    sub = points.iloc[i * points_per_subset:(i + 1) * points_per_subset].copy()
    
    # Add the start and end points to the subset
    sub = pd.concat([sub, start.to_frame().T, end.to_frame().T], ignore_index=True)
    
    # Save the subset to a CSV file
    filepath = output_dir / f"{subset_name}{i}.csv"
    sub.to_csv(filepath, index=False)

print(f"{subsets} subsets saved to {output_dir}")

20 subsets saved to subsets


In [59]:
path_points = pd.DataFrame(columns=["x", "y", "z"])
path_points = pd.concat([path_points, start.to_frame().T], ignore_index=True, axis=0)
# Define the heuristic function for A*
def heuristic1(node1, node2):
    x1, y1, z1 = G.nodes[node1]['x'], G.nodes[node1]['y'], G.nodes[node1]['z']
    x2, y2, z2 = G.nodes[node2]['x'], G.nodes[node2]['y'], G.nodes[node2]['z']
    return heur_dist(x1, y1, z1, x2, y2, z2)

for k in range(subsets):
    sub = pd.read_csv(os.path.join(subsets_path, f"{subset_name}{k}.csv"))

    G = nx.Graph()

    # Add nodes from subset
    for i, row in sub.iterrows():
        G.add_node(i, x = row['x'], y = row['y'], z = row['z'])

    # Add edges between the closest nodes (ignoring height)
    coords = sub[['x', 'y']].values
    tree = spatial.KDTree(coords)
    for i, row in sub.iterrows():
        distances, indices = tree.query([row['x'], row['y']], k=depth+1)
        for j in range(1, len(indices)):  # skip the first index because it is the point itself
            G.add_edge(i, indices[j], weight=distances[j])

    # check if the graph is connected
    if not nx.is_connected(G):
        # start at a node. add an edge to the closest node that is not connected
        # repeat until the graph is connected
        start_node = sub.shape[0] - 2
        end_node = sub.shape[0] - 1
        while not nx.has_path(G, start_node, end_node):
            # find the closest node that is not connected
            for i, row in sub.iterrows():
                if not nx.has_path(G, start_node, i):
                    w = math.sqrt((sub.loc[start_node, 'x'] - sub.loc[i, 'x'])**2 + (sub.loc[start_node, 'y'] - sub.loc[i, 'y'])**2)
                    G.add_edge(start_node, i, weight=w)
                    break

    # Find the shortest path using A*
    path = nx.astar_path(G, sub.shape[0] - 2, sub.shape[0] - 1, heuristic=heuristic1)
    path = pd.Series(path)
    
    # add path to path_points
    path_points = pd.concat([path_points, sub.loc[path]], ignore_index=True, axis=0)

print("\nDone")

/var/folders/nj/3217hbg559b0blsl1jfx56tw0000gn/T/ipykernel_90092/839333947.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  path_points = pd.concat([path_points, start.to_frame().T], ignore_index=True, axis=0)



Done


In [61]:
path_points.drop_duplicates(inplace=True, keep='first')
path_points.reset_index(drop=True, inplace=True)
# add end point to path_points
path_points = pd.concat([path_points, end.to_frame().T], ignore_index=True, axis=0)
path_points.to_csv("path_points.csv", index=False)
print(path_points)

           x       y      z
0    1457.45   44.38   9.84
1    1413.17   22.41  10.37
2    1358.38   20.72   8.69
3    1318.60   41.02   8.47
4    1278.79   88.57   8.66
..       ...     ...    ...
742   427.78  802.72  34.77
743   415.33  824.43  38.74
744   402.32  848.67  36.74
745   403.11  872.12  43.25
746   424.20  910.96  36.22

[747 rows x 3 columns]


In [42]:
print("Finding path for path_points")
# make new graph using path_points
G = nx.Graph()

# Define the heuristic function for A*
def heuristic2(node1, node2):
    x1, y1, z1 = G.nodes[node1]['x'], G.nodes[node1]['y'], G.nodes[node1]['z']
    x2, y2, z2 = G.nodes[node2]['x'], G.nodes[node2]['y'], G.nodes[node2]['z']
    return heur(x1, y1, z1, x2, y2, z2)

# Add nodes from path_points
for i, row in path_points.iterrows():
    G.add_node(i, x = row['x'], y = row['y'], z = row['z'])

# Add edges between the closest nodes
coords = path_points[['x', 'y', 'z']].values
tree = spatial.KDTree(coords)
for i, row in path_points.iterrows():
    distances, indices = tree.query([row['x'], row['y'], row['z']], k=depth+1)
    for j in range(1, len(indices)):  # skip the first index because it is the point itself
        node1 = i
        node2 = indices[j]
        G.add_edge(node1, node2, weight=heuristic2(node1, node2))

# check if the graph is connected
if not nx.is_connected(G):
    print(f"Graph is not connected. Adding edges to connect the graph")
    # start at a node. add an edge to the closest node that is not connected
    # repeat until the graph is connected
    start_node = path_points.shape[0] - 2
    end_node = path_points.shape[0] - 1
    while not nx.has_path(G, start_node, end_node):
        # find the closest node that is not connected
        for i, row in path_points.iterrows():
            if not nx.has_path(G, start_node, i):
                w = heuristic2(start_node, i)
                G.add_edge(start_node, i, weight=w)
                break

# Find the shortest path using A*
path_i = nx.astar_path(G, 0, path_points.shape[0] - 1, heuristic=heuristic2)
print("\nDone")

Finding path for path_points

Done


In [62]:
path_i = pd.Series(path_i)
path_i = np.ndarray.flatten(path_i.to_numpy())
path = path_points.loc[path_i]
path = path.to_numpy()
print(path)

[[1457.45   44.38    9.84]
 [1436.17   59.01    9.52]
 [1419.49   53.17    9.45]
 [1398.97   66.91    9.13]
 [1392.46   70.25    8.98]
 [1393.17  100.63    9.3 ]
 [1362.74   99.14    8.42]
 [1340.69  113.25    8.85]
 [1324.08  132.65    8.83]
 [1314.99  129.18    8.75]
 [1302.94  131.      8.42]
 [1283.24  137.73    7.94]
 [1261.51  130.01    7.8 ]
 [1249.45  122.11    8.17]
 [1232.82  123.66    8.02]
 [1208.04  119.25    8.06]
 [1196.47  116.78    8.04]
 [1168.91  122.49    7.88]
 [1141.72  131.35    7.74]
 [1127.27  116.2     7.77]
 [1078.24  115.38    7.47]
 [1029.46  134.03    6.79]
 [ 980.19  149.15    4.6 ]
 [ 917.29  195.37    4.52]
 [ 892.23  193.38    4.65]
 [ 812.18  153.98    3.9 ]
 [ 777.69  251.03    3.85]
 [ 725.62  334.4     4.7 ]
 [ 684.58  415.04    5.47]
 [ 612.5   441.87    7.39]
 [ 578.27  534.      8.27]
 [ 588.11  555.92    8.22]
 [ 584.62  596.51    9.59]
 [ 587.62  632.57   10.38]
 [ 579.2   663.39   11.72]
 [ 586.99  684.27   13.19]
 [ 570.63  697.76   18.2 ]
 